In [106]:
from openai import OpenAI

In [107]:
from dotenv import load_dotenv

In [108]:
from supabase import create_client, Client

In [109]:
import anthropic

In [110]:
load_dotenv()

True

In [111]:
import os

In [112]:
import requests

In [114]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [115]:
anthropic_api_key = os.getenv("CLAUDE_API_KEY")

In [136]:
client_anthropic = anthropic.Anthropic(api_key = anthropic_api_key)

In [22]:
supabase_url = os.environ.get("SUPABASE_URL")

In [23]:
supabase_api_key = os.environ.get("SUPBASE_KEY")

In [25]:
supabase: Client = create_client(supabase_url, supabase_api_key)

In [27]:
sql_query

'\nSELECT * FROM webhooks order by created_at ASC\n'

In [53]:
result = supabase.table('webhooks')\
    .select('id, news_output,source_urls')\
    .order('created_at', desc=False)\
    .execute()

result.data

[{'id': 2,
  'news_output': 'Reuters (Jan 24, 2026) reports the Singapore government will invest over S$1 billion (about $778.8 million) in public AI research through 2030. The funding targets priority research areas including responsible and resource-efficient AI, talent development from pre-university to faculty levels, and building capabilities to support industry adoption of AI. The announcement builds on prior Singapore commitments (e.g., 2024 HPC funding and AI Singapore investments) and is intended to strengthen national AI capabilities and global competitiveness.',
  'source_urls': ['https://www.reuters.com/world/asia-pacific/singapore-invest-over-779-million-public-ai-research-through-2030-2026-01-24/']},
 {'id': 3,
  'news_output': "TechCrunch (Jan 24, 2026) published an analysis titled 'A new test for AI labs: Are you even trying to make money?' The piece proposes a five-level scale to rate AI labs by commercial ambition and summarizes recent funding and personnel developmen

In [168]:
def prompt_generator_research():
    formatted_prompt = []
    for item in result.data:
        formatted_prompt.append(
            f"""You are given news summary: {item['news_output']} and source: {item['source_urls'][0]}, you are supposed to run a deep research and read other sources as well. And then give a well structured ouput in the form of points and sources."""
        )

    return [formatted_prompt[0]]

In [169]:
prompt_generator_research()

['You are given news summary: Reuters (Jan 24, 2026) reports the Singapore government will invest over S$1 billion (about $778.8 million) in public AI research through 2030. The funding targets priority research areas including responsible and resource-efficient AI, talent development from pre-university to faculty levels, and building capabilities to support industry adoption of AI. The announcement builds on prior Singapore commitments (e.g., 2024 HPC funding and AI Singapore investments) and is intended to strengthen national AI capabilities and global competitiveness. and source: https://www.reuters.com/world/asia-pacific/singapore-invest-over-779-million-public-ai-research-through-2030-2026-01-24/, you are supposed to run a deep research and read other sources as well. And then give a well structured ouput in the form of points and sources.']

In [182]:
system_prompt = prompt_generator_research()

In [180]:
def openai_research():
    research_output = []
    for i in system_prompt:
        print(i)
        response = client.responses.create(
            model = "gpt-4o",
            # reasoning = {"effort": "high"},
            tools = [{
                "type": "web_search"
            }],

            include = ["web_search_call.action.sources"],
            input = i,
            # max_output_tokens = 1024
        )

        output = response.output_text

        research_output.append(output)

    return research_output

In [181]:
openai_research()

You are given news summary: Reuters (Jan 24, 2026) reports the Singapore government will invest over S$1 billion (about $778.8 million) in public AI research through 2030. The funding targets priority research areas including responsible and resource-efficient AI, talent development from pre-university to faculty levels, and building capabilities to support industry adoption of AI. The announcement builds on prior Singapore commitments (e.g., 2024 HPC funding and AI Singapore investments) and is intended to strengthen national AI capabilities and global competitiveness. and source: https://www.reuters.com/world/asia-pacific/singapore-invest-over-779-million-public-ai-research-through-2030-2026-01-24/, you are supposed to run a deep research and read other sources as well. And then give a well structured ouput in the form of points and sources.


["Here is a **well-structured, in-depth analysis** (approximately 900 words), summarizing and adding context through multiple reputable sources, with each statement duly cited. I present the details in clear bullet-point sections for readability, as requested:\n\n---\n\n### • Overview of the Announcement\n\n- On **January 24, 2026**, Reuters reported that the **Singapore government will commit over S$1\u202fbillion (approximately US$778.8\u202fmillion)** to support **public AI research through 2030**, aiming to enhance national AI capabilities and global competitiveness ([nst.com.my](https://www.nst.com.my/amp/world/region/2026/01/1363738/singapore-invest-over-us779-million-public-ai-research-through-2030?utm_source=openai)).\n- Government statements clarify this sum is allocated under the **National AI Research and Development Plan (NAIRD)** spanning **2025 to 2030**, and is part of broader efforts under the updated **National AI Strategy (NAIS 2.0)** ([channelnewsasia.com](https://ww

In [101]:
def openai_article_builder(researched_data):
    final_news_article = []
    for i in researched_data:
        response = client.responses.create(
            model = "gpt-4.1",
            input = f""" 
            You are a professional tech journalist for "The AI Times" newspaper.

            Your task: Write a comprehensive news article covering today's AI developments.
            
            Entire news story and its details: {i}
            
            INSTRUCTIONS:
            1. Write a newspaper-quality article (800-1000 words)
            2. Create an engaging headline
            3. Start with a lead paragraph summarizing the most important developments
            5. Use journalistic style: objective, clear, professional and opinionated
            6. Include all the sources & links at the end and not ANY sources or links during the article

            You are ONLY supposed to refer to the sources and links and NOT supposed to hallucinate. 
            
            However, the opinion you have be your own but it should be based on the above article. 
            """
        )


        final_output = response.output_text

        print(final_output)

        final_news_article.append(final_output)

    return final_news_article

In [230]:
def anthropic_research():
    research_ouput_anthropic = []
    for i in system_prompt:
        response = client_anthropic.messages.create(
            model = "claude-haiku-4-5-20251001",
            max_tokens = 4096,
            messages = [
                {
                    "role": "user",
                    "content": f"{i}"
                }
            ],
            tools = [
                {
                    "type": "web_search_20250305",
                    "name": "web_search",
                    "max_uses": 5
                }
            ]
        )

        research_ouput_anthropic.append(response.content)

    return research_ouput_anthropic

In [231]:
def extract_final_text(blocks):
    last_tool_idx = -1
    for i,b in enumerate(blocks):
        if getattr(b, "type", None) in ("server_tool_use", "web_search_tool_result"):
            last_tool_idx = i

    text_parts = []
    for b in blocks[last_tool_idx + 1:]:
        if getattr(b, "type", None) == "text":
            text_parts.append(b.text)

    return "\n".join(text_parts).strip()

In [232]:
results = anthropic_research()

for idx, blocks in enumerate(results, 1):
    print(extract_final_text(blocks))

Based on my comprehensive research, here's a well-structured analysis of Singapore's S$1 billion AI research investment and related initiatives:

---

## **SINGAPORE'S S$1 BILLION AI RESEARCH INVESTMENT (2026-2030)**

### **I. PRIMARY ANNOUNCEMENT & FUNDING DETAILS**


The Singapore government will invest over S$1 billion ($778.8 million) in public artificial intelligence research through 2030 to strengthen the nation's capabilities and global competitiveness
.

**Key Allocation Areas:**
- 
Building responsible and resource-efficient AI, and developing the nation's talent from pre-university to faculty

- 
Building capabilities to support the adoption and application of AI by industries

- 
Building responsible, trustworthy and resource-efficient AI systems


**Sources:**
- Reuters (Jan 24, 2026) - Primary announcement
- Brandi Icon Image, KELO-AM, The Sunday Guardian, Yahoo News, Computer Weekly

---

### **II. PRIOR GOVERNMENT INVESTMENTS IN AI**

**2024 High-Performance Computing (H